# IBM - Banks ETL Pipline

## Context

***Scenario:***<br>
You have been hired by research organization. You have been tasked to create a code that can be used to compile the list of the top 10 largest banks in the world ranked by market capitalization in billion USD. Further, the data needs to be transformed and stored in GBP, EUR and INR as well, in accordance with the exchange rate information. The processed information table is to be saved locally in a CSV format and as a database table.<br>
The task is to create an automated system to generate this information so that the same can be executed in every financial quarter to prepare the report.
<br>

**Task 1:**<br>
Write a function log_progress() to log the progress of the code at different stages in a file code_log.txt. Use the list of log points provided to create log entries as every stage of the code.
<br><br>
**Task 2:**<br>
Extract the tabular information from the given URL under the heading 'By market capitalization' and save it to a dataframe.
a. Inspect the webpage and identify the position and pattern of the tabular information in the HTML code
b. Write the code for a function extract() to perform the required data extraction.
c. Execute a function call to extract() to verify the output.
<br><br>
**Task 3:**<br>
Transform the dataframe by adding columns for Market Capitalization in GBP, EUR and INR, rounded to 2 decimal places, based on the exchange rate information shared as a CSV file.
a. Write the code for a function transform() to perform the said task.
b. Execute a function call to transform() and verify the output.
<br><br>
**Task 4:**<br>
Load the transformed dataframe to an output CSV file. Write a function load_to_csv(), execute a function call and verify the output.
<br><br>
**Task 5:**<br>
Load the transformed dataframe to an SQL database server as a table. Write a function load_to_db(), execute a function call and verify the output.
<br><br>
**Task 6:**<br>
Run queries on the database table. Write a function load_to_db(), execute a given set of queries and verify the output.
<br><br>
**Task 7:**<br>
Verify that the log entries have been completed at all stages by checking the contents of the file code_log.txt.

-------

## Set Up

In [12]:
# Import dependencies
import pandas as pd
import numpy as np
from datetime import datetime
import requests
import sqlite3
from bs4 import BeautifulSoup

In [13]:
# Accessing the required data
# The below is the url where the exchange rate data can be found
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv

# The below is the url where the 'largest banks' data can be found
# https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks

## Task 1

In [14]:
def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''
    
    # Creating the current time and parsing into desired string format
    time_now = datetime.now().strftime('%Y-%h-%d-%H:%M:%S')
    
    # Append timestamp and message to process log file
    with open('code_log.txt', 'a') as file:
        file.write(f'{time_now} : {message}' + '\n')

In [15]:
# Creating the first log entry
log_progress("Preliminaries complete. Initiating ETL process")

## Task 2

### Figuring Out the Logic

In [16]:
# Requesting wiki web page
url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
page = requests.get(url)

# Creating a soup object of the wiki page
soup = BeautifulSoup(page.text)

In [17]:
# Finding the required table
tables = soup.find_all('table')

market_cap_tb = tables[0]
market_cap_tb.find_all('tr')[10].find_all('td')[0].text

'10\n'

In [18]:
# Empty list to be populated with several other lists. Each of these lists will represent
# a row in the wiki table
extracted_data = []

# Iterate through each row in the market cap table and create an empty list
for row in market_cap_tb.find_all('tr'):
    extract_row = []
    
    # Iterate through each cell in the row and append the cell contents to a list
    for cell in row.find_all('td'):
        extract_row.append(cell.text[:-1]) # Each cell ends in "\n" so [:-1] will ignore the last two characters 
    
    # Append the list of cell contents to the extracted_data list
    extracted_data.append(extract_row)
        
    
extracted_data

[[],
 ['1', ' JPMorgan Chase', '432.92'],
 ['2', ' Bank of America', '231.52'],
 ['3', ' Industrial and Commercial Bank of China', '194.56'],
 ['4', ' Agricultural Bank of China', '160.68'],
 ['5', ' HDFC Bank', '157.91'],
 ['6', ' Wells Fargo', '155.87'],
 ['7', ' HSBC Holdings PLC', '148.90'],
 ['8', ' Morgan Stanley', '140.83'],
 ['9', ' China Construction Bank', '139.82'],
 ['10', ' Bank of China', '136.81']]

In [19]:
# As the first row of the wiki table contains table headers, "find_all('td')" will return empty
# This will replace the first empty list in 'extracted_data' with the table headers
table_headers = [header.text[:-1] for header in market_cap_tb.find_all('th')]
extracted_data[0] = table_headers
extracted_data

[['Rank', 'Bank name', 'Market cap(US$ billion)'],
 ['1', ' JPMorgan Chase', '432.92'],
 ['2', ' Bank of America', '231.52'],
 ['3', ' Industrial and Commercial Bank of China', '194.56'],
 ['4', ' Agricultural Bank of China', '160.68'],
 ['5', ' HDFC Bank', '157.91'],
 ['6', ' Wells Fargo', '155.87'],
 ['7', ' HSBC Holdings PLC', '148.90'],
 ['8', ' Morgan Stanley', '140.83'],
 ['9', ' China Construction Bank', '139.82'],
 ['10', ' Bank of China', '136.81']]

In [20]:
# Creating a pandas dataframe using extracted data
df = pd.DataFrame(extracted_data)

# Set first row as column names and drop that row
df.columns = df.loc[0, :]
df = df.drop(index=0)

# Dropping the Rank column
df = df.drop(columns='Rank')

# Renaming columns
df.columns = ['Name', 'MC_USD_Billion']

# Type cast the market cap column to float
df['MC_USD_Billion'] = df['MC_USD_Billion'].astype(float)

df

,Name,MC_USD_Billion
1,JPMorgan Chase,432.92
2,Bank of America,231.52
3,Industrial and Commercial Bank of China,194.56
4,Agricultural Bank of China,160.68
5,HDFC Bank,157.91
6,Wells Fargo,155.87
7,HSBC Holdings PLC,148.90
8,Morgan Stanley,140.83
9,China Construction Bank,139.82
10,Bank of China,136.81


### Building Function and Calling It

In [21]:
def extract(url='https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks', 
            table_attribs=['Name', 'MC_USD_Billion']):
    
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''
    
    # Requesting wiki web page
    url = url
    page = requests.get(url)

    # Creating a soup object of the wiki page
    soup = BeautifulSoup(page.text)
    
    # Finding the required table
    tables = soup.find_all('table')
    market_cap_tb = tables[0]    
    
    # Empty list to be populated with several other lists. Each of these lists will represent
    # a row in the wiki table
    extracted_data = []

    # Iterate through each row in the market cap table and create an empty list
    for row in market_cap_tb.find_all('tr'):
        extract_row = []

        # Iterate through each cell in the row and append the cell contents to a list
        for cell in row.find_all('td'):
            extract_row.append(cell.text[:-1]) # Each cell ends in "\n" so [:-1] will ignore the last two characters 

        # Append the list of cell contents to the extracted_data list
        extracted_data.append(extract_row)
    
    
    # As the first row of the wiki table contains table headers, "find_all('td')" will return empty
    # This will replace the first empty list in 'extracted_data' with the table headers
    table_headers = [header.text[:-1] for header in market_cap_tb.find_all('th')]
    extracted_data[0] = table_headers
    
    # Creating a pandas dataframe using extracted data
    df = pd.DataFrame(extracted_data)

    # Set first row as column names and drop that row
    df.columns = df.loc[0, :]
    df = df.drop(index=0)

    # Dropping the Rank column
    df = df.drop(columns='Rank')

    # Renaming columns
    df.columns = table_attribs

    # Type cast the market cap column to float
    df['MC_USD_Billion'] = df['MC_USD_Billion'].astype(float)
    
    # Log etl progress
    log_progress("Data extraction complete. Initiating Transformation process")
    
    
    return df

In [22]:
# Calling extract function
extract()

,Name,MC_USD_Billion
1,JPMorgan Chase,432.92
2,Bank of America,231.52
3,Industrial and Commercial Bank of China,194.56
4,Agricultural Bank of China,160.68
5,HDFC Bank,157.91
6,Wells Fargo,155.87
7,HSBC Holdings PLC,148.90
8,Morgan Stanley,140.83
9,China Construction Bank,139.82
10,Bank of China,136.81


## Task 3

### Figuring Out the Logic

In [23]:
# Read exchange rate csv file
er_df = pd.read_csv('data/exchange_rate.csv')
er_df.head()

,Currency,Rate
0,EUR,0.93
1,GBP,0.80
2,INR,82.95


In [24]:
# Convert dataframe into a dictionairy where the first column represents the keys and the second column represents the values
er_dict = {}
for currency, rate in zip(er_df['Currency'], er_df['Rate']):
    er_dict[currency] = rate
    
er_dict

{'EUR': 0.93, 'GBP': 0.8, 'INR': 82.95}

In [25]:
# Adding exchange rate adjusted columns to the dataframe
df['MC_GBP_Billion'] = round(df['MC_USD_Billion'] * er_dict['GBP'], 2)
df['MC_EUR_Billion'] = round(df['MC_USD_Billion'] * er_dict['EUR'], 2)
df['MC_INR_Billion'] = round(df['MC_USD_Billion'] * er_dict['INR'], 2)

df

,Name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
1,JPMorgan Chase,432.92,346.34,402.62,35910.71
2,Bank of America,231.52,185.22,215.31,19204.58
3,Industrial and Commercial Bank of China,194.56,155.65,180.94,16138.75
4,Agricultural Bank of China,160.68,128.54,149.43,13328.41
5,HDFC Bank,157.91,126.33,146.86,13098.63
6,Wells Fargo,155.87,124.70,144.96,12929.42
7,HSBC Holdings PLC,148.90,119.12,138.48,12351.26
8,Morgan Stanley,140.83,112.66,130.97,11681.85
9,China Construction Bank,139.82,111.86,130.03,11598.07
10,Bank of China,136.81,109.45,127.23,11348.39


### Building Function and Calling It

In [26]:
def transform(df, csv_path='data/exchange_rate.csv'):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''
    
    # Read exchange rate csv file
    er_df = pd.read_csv(csv_path)
    
    # Convert dataframe into a dictionairy where the first column represents the keys and the second column represents the values
    er_dict = {}
    for currency, rate in zip(er_df['Currency'], er_df['Rate']):
        er_dict[currency] = rate
    
    # Adding exchange rate adjusted columns to the dataframe
    df['MC_GBP_Billion'] = round(df['MC_USD_Billion'] * er_dict['GBP'], 2)
    df['MC_EUR_Billion'] = round(df['MC_USD_Billion'] * er_dict['EUR'], 2)
    df['MC_INR_Billion'] = round(df['MC_USD_Billion'] * er_dict['INR'], 2)

    # Log etl progress
    log_progress("Data transformation complete. Initiating Loading process")
    
    
    return df

In [27]:
# Calling transform function
transform(extract())

,Name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
1,JPMorgan Chase,432.92,346.34,402.62,35910.71
2,Bank of America,231.52,185.22,215.31,19204.58
3,Industrial and Commercial Bank of China,194.56,155.65,180.94,16138.75
4,Agricultural Bank of China,160.68,128.54,149.43,13328.41
5,HDFC Bank,157.91,126.33,146.86,13098.63
6,Wells Fargo,155.87,124.70,144.96,12929.42
7,HSBC Holdings PLC,148.90,119.12,138.48,12351.26
8,Morgan Stanley,140.83,112.66,130.97,11681.85
9,China Construction Bank,139.82,111.86,130.03,11598.07
10,Bank of China,136.81,109.45,127.23,11348.39


## Task 4

### Figuring Out the Logic

### Building Function and Calling It

In [33]:
def load_to_csv(df, output_path='./Largest_banks_data.csv'):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''
    
    # Save dataframe as csv file
    df.to_csv(output_path)
    
    # Log etl progress
    log_progress("Data saved to CSV file")

In [34]:
# load_to_csv(transform(extract()))

## Task 5

### Figuring Out the Logic

### Building Function and Calling It

In [ ]:
def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''

## Task 6

### Figuring Out the Logic

### Building Function and Calling It

In [ ]:
def run_query(query_statement, sql_connection):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''

## Task 7

### Figuring Out the Logic

### Building Function and Calling It

In [3]:
''' Here, you define the required entities and call the relevant
functions in the correct order to complete the project. Note that this
portion is not inside any function.'''

None